In [9]:
import requests
import warnings
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from langdetect import detect
from selenium import webdriver
import os
from googlesearch import search
import time


warnings.filterwarnings("ignore", message="Unverified HTTPS request")
driver = webdriver.Chrome()

# pip install requests
# pip install beautifulsoup4
# pip install pandas
# pip install langdetect
# pip install google


In [14]:
def removeHtmlTags(htmlText):
    soup = BeautifulSoup(htmlText, 'html.parser')
    text = soup.get_text(separator=" ", strip=True)
    words = text.split()
    # Filter out words containing "https://"
    filtered_words = [word for word in words if "https://" not in word and "http://" not in word]
    filtered_text = ' '.join(filtered_words)
    return filtered_text


def extractText(url):
    res = requests.get(url, verify=False)
    if res.status_code == 200:
        htmlPage = res.content
        soup = BeautifulSoup(htmlPage, 'html.parser')
        visibleText = soup.get_text()
        visibleText = removeHtmlTags(visibleText)
        lines = visibleText.split('\n')
        # i check for '.' and for a line longer that 200 chars to eliminate titles, headers
        text = '\n'.join(line.strip() for line in lines if len(line.strip()) > 0 and len(line) > 200 and '.' in line)
        return text
    return None

def canTextBeExtracted(url):
    res = requests.get(url, verify=False)
    if res.status_code == 200:
        htmlPage = res.content
        soup = BeautifulSoup(htmlPage, 'html.parser')
        visibleText = soup.get_text()
        lines = visibleText.split('\n')
        text = ' '.join(line.strip() for line in lines if len(line.strip()) > 0 and len(line) > 200 and '.' in line)
        text = removeHtmlTags(text)
        # i choose articles longer than 500 chars
        if len(text) > 500 and detect(text) == 'ro':
            return True
    return False

from urllib.parse import urlparse, urljoin

def findLinks(url, sitesToVisit, visitedSites):
    base_domain = urlparse(url).netloc.lower()
    blacklist = ['youtube.com', 'instagram.com', 'pinterest.com', 'twitter.com', 'facebook.com',
                 'login', 'cookie', 'cookies', 'politica-de-confidentialitate', 'despre-noi', 
                 'termeni-si-conditii', 'contact', 'privacy-policy', 'search', 'archive', 
                 'tag', 'category', 'forum', 'login', 'register', 'profile', 'logout', 
                 'sign-up','log-in','my-account','privacy','conditii','service','terms','comment','comentariu','respond',
                 'conditions','about','sitemap','cont','comments','feed','politica-editoriala','cum-ne-poti-ajuta',
                 'password','paywall','arhiva','archive','termeni','despre','admin','newsletter',
                 'cart', 'checkout', 'shop', 'store', 'download', 'subscribe', 'unsubscribe','produs','abonare',
                 'terms-of-service', 'about-us', 'faq', 'donate', 'events', 'calendar', 
                 'faq', 'gallery', 'help', 'guidelines', 'policy']
    res = requests.get(url, verify=False)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        navbar = soup.find('nav', class_='navbar')
        footer = soup.find('footer', class_='footer')
        links = soup.find_all('a', href=True)
        for link in links:
            if navbar and link in navbar.find_all('a') or footer and link in footer.find_all('a'):
                continue
            href = link['href']
            if href.startswith('#') or href.startswith('javascript:'):
                continue  
            currentLink = urljoin(url, href)
            current_domain = urlparse(currentLink).netloc.lower()
            if current_domain == base_domain:
                if currentLink not in visitedSites and currentLink not in sitesToVisit and all(b not in currentLink.lower() for b in blacklist):
                    sitesToVisit.append(currentLink)
        return 0
    return 1

def extractAll(nrOfSitesToVisit, url, file_name,siteIdx):
    data = pd.DataFrame(columns=['siteIdx','link','Text'])
    sitesToVisit = []
    visitedSites = []
    initialSitesToVisit = nrOfSitesToVisit
    tries = 1
    findLinksCode = findLinks(url, sitesToVisit, visitedSites)
    startTime = time.time()
    if findLinksCode == 0:
        while nrOfSitesToVisit:
            if sitesToVisit:
                link = sitesToVisit[-1]
                extractableText = canTextBeExtracted(link)
                if extractableText:
                    text = extractText(link)
                    if text:
                        #succesful text extraction
                        new_row = {'siteIdx':siteIdx, 'link': link, 'Text': text}
                        data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
                        nrOfSitesToVisit -= 1
                        startTime = time.time()
                visitedSites.append(link)
                sitesToVisit.pop()

                print("Visited site: ",link)
            elif nrOfSitesToVisit:
                if len(visitedSites) > tries: 
                    findLinksCode = findLinks(visitedSites[tries], sitesToVisit, visitedSites)
                    tries += 1
                else:
                    print('Only', initialSitesToVisit - nrOfSitesToVisit, 'articles out of', initialSitesToVisit, 'for', url)
                    nrOfSitesToVisit = 0
            endTime = time.time()
            if endTime - startTime > 180:
                print("1 minute time limit exceeded, ",tries," unsuccesful tries")
                if not os.path.isfile(file_name):
                    data.to_csv(file_name, index=False)
                else:
                    data.to_csv(file_name, index=False, mode='a', header=False)
                return
    if not os.path.isfile(file_name):
        data.to_csv(file_name, index=False)
    else:
        data.to_csv(file_name, index=False, mode='a', header=False)
    return



def processUrls(nrOfSitesToVisit,urls, fileName):

    with open(fileName, 'w'):
        pass

    allData = pd.DataFrame(columns=['siteIdx','link', 'Text'])
    siteIdx = 0
    for url in urls:
        try:
            siteIdx += 1
            data = extractAll(nrOfSitesToVisit, url,fileName,siteIdx)
            allData = pd.concat([allData, data], ignore_index=True) 
        except Exception as e:
            print(f"Error occurred while processing {url}: {e}")
    return allData


In [12]:
parentingBlogsMd = [
"https://taticulmodern.com/",
"https://www.planetamami.com/",
"https://suntmama.com/",
"https://anairina.com/",
"https://prietenamea.com/",
"https://copilariamamicilor.wordpress.com/",
"https://sunttata.blogspot.com/search/label/Blog",
"https://miciidescoperitori.blogspot.com/",
"https://jurnaldetatic.blogspot.com/",
"https://vulpea.blog/category/maternitate/"
]

parentingArticles = processUrls(100,parentingBlogsMd,'parentingMd.csv')

Visited site:  https://www.planetamami.com/remedii-naturale-raceala-la-copii
Visited site:  https://www.planetamami.com/activitati-pentru-copii-de-1-an
Visited site:  https://www.planetamami.com/solutii-naturale-pentru-calmarea-tusei-la-copii
Visited site:  https://www.planetamami.com/35-de-activitati-pentru-copii-de-18-luni
Visited site:  https://www.planetamami.com/activitati-pentru-copii-2-3-ani
Visited site:  http://www.planetamami.com/remedii-naturale-raceala-la-copii
Visited site:  http://www.planetamami.com/pastreaza-calmul-cand-copilul-tau-are-febra
Visited site:  https://www.planetamami.com/page/20
Visited site:  https://www.planetamami.com/page/2
Visited site:  https://www.planetamami.com/schimbari-de-comportament-la-copii-in-pandemie
Visited site:  https://www.planetamami.com/colectie-de-jocuri-si-activitati-in-casa
Visited site:  https://www.planetamami.com/reteta-vegan-banana-bread
Visited site:  https://www.planetamami.com/activitate-de-familie-gatim-biscuitii-de-craciun


C:\Users\ilinc\AppData\Local\Temp\ipykernel_18076\2201444195.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(htmlText, 'html.parser')


Visited site:  https://suntmama.com/2015/09/
Visited site:  https://suntmama.com/2015/10/
Visited site:  https://suntmama.com/2015/11/
Visited site:  https://suntmama.com/2015/12/
Visited site:  https://suntmama.com/2016/01/
Visited site:  https://suntmama.com/2016/02/
Visited site:  https://suntmama.com/2016/03/
Visited site:  https://suntmama.com/2016/04/
Visited site:  https://suntmama.com/2016/05/
Visited site:  https://suntmama.com/2016/06/
Visited site:  https://suntmama.com/2016/07/
Visited site:  https://suntmama.com/2016/08/
Visited site:  https://suntmama.com/2016/09/
Visited site:  https://suntmama.com/2016/10/
Visited site:  https://suntmama.com/2016/11/
Visited site:  https://suntmama.com/2016/12/
Visited site:  https://suntmama.com/2017/02/
Visited site:  https://suntmama.com/2017/03/
Visited site:  https://suntmama.com/2017/04/
Visited site:  https://suntmama.com/2017/05/
Visited site:  https://suntmama.com/2017/06/
Visited site:  https://suntmama.com/2017/07/
Visited si

In [15]:
sportBlogsMd = [
"https://blog.antrenor.md/",
"https://sandugrecu.blogspot.com/",
"https://acvilasport.md/blog",
"https://enersport.md/blog/",
"https://fmf.md/blog",
"https://dinamo.md/blog/",
"https://www.ellefitness.md/ro/blog",
"https://unica.md/sport/"
]

sportArticlesMd = processUrls(100,sportBlogsMd,'sportMd.csv')

Visited site:  https://blog.antrenor.md/posts/antrenament-ideal-pentru-rezultate-bune
Visited site:  https://blog.antrenor.md/posts/metoda-ideala-de-antrenament
Visited site:  https://blog.antrenor.md/posts/exercitii-pentru-arderea-grasimii-de-pe-burta
Visited site:  https://blog.antrenor.md/posts/cele-mai-eficiente-exercitii-pentru-un-abdomen-plat
Visited site:  https://blog.antrenor.md/posts/antrenament-de-20-minute-pentru-picioare-senzationale
Visited site:  https://blog.antrenor.md/posts/cum-sa-supravietuiesti-unui-atac-de-cord-cind-esti-singur
Visited site:  https://blog.antrenor.md/posts/tu-cum-te-motivezi-cand-nu-ai-chef-de-sport
Visited site:  https://blog.antrenor.md/posts/povestea-lui-lee-haney-detinatorul-a-8-titluri-mr-olympia
Visited site:  https://blog.antrenor.md/posts/febra-musculara
Visited site:  https://blog.antrenor.md/posts/sfaturi-pentru-incepatoare-primele-exercitii-cu-greutati
Visited site:  https://blog.antrenor.md/posts/dietele-nu-dau-rezultate-daca-faci-acest

C:\Users\ilinc\AppData\Local\Temp\ipykernel_18076\4105644542.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(htmlText, 'html.parser')


Visited site:  https://acvilasport.md/fitness/haltere-si-greutati/haltere-greutati-si-bare
Visited site:  https://acvilasport.md/fitness/haltere-si-greutati/gantere
Visited site:  https://acvilasport.md/fitness/haltere-si-greutati/scaun-pentru-haltere
Visited site:  https://acvilasport.md/fitness/haltere-si-greutati
Visited site:  https://acvilasport.md/fitness/aparate-fitnes/accesorii-pentru-aparate-fitnes
Visited site:  https://acvilasport.md/fitness/aparate-fitnes/scaun-pentru-exercitii
Visited site:  https://acvilasport.md/fitness/aparate-fitnes/banda-de-alergat
Visited site:  https://acvilasport.md/fitness/aparate-fitnes/aparat-multifunctional
Visited site:  https://acvilasport.md/fitness/aparate-fitnes
Visited site:  https://acvilasport.md/fitness/fitness/aparatori-si-protectii-suporturi-corporale-fitnes
Visited site:  https://acvilasport.md/fitness/fitness/greutati-gantere-haltere-si-bare-fitnes
Visited site:  https://acvilasport.md/fitness/fitness/step-fitnes
Visited site:  htt

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://fmf.md/cdn/blog/7533/DSC_0516.JPG


...

Visited site:  https://fmf.md/cdn/blog/7533/DSC_0501.JPG


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://fmf.md/files/docs/blog/UEFA SROI MDA trad ro.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Only 13 articles out of 100 for https://fmf.md/blog
Visited site:  https://dinamo.md/2016/12/
Visited site:  https://dinamo.md/2017/02/
Visited site:  https://dinamo.md/2017/03/
Visited site:  https://dinamo.md/2017/04/
Visited site:  https://dinamo.md/2017/05/
Visited site:  https://dinamo.md/2017/06/
Visited site:  https://dinamo.md/2017/07/
Visited site:  https://dinamo.md/2017/08/
Visited site:  https://dinamo.md/2017/09/
Visited site:  https://dinamo.md/2017/11/
Visited site:  https://dinamo.md/2018/01/
Visited site:  https://dinamo.md/2018/02/
Visited site:  https://dinamo.md/2018/03/
Visited site:  https://dinamo.md/2018/04/
Visited site:  https://dinamo.md/2018/05/
Visited site:  https://dinamo.md/2018/06/
Visited site:  https://dinamo.md/2018/07/
Visited site:  https://dinamo.md/2018/09/
Visited site:  https://dinamo.md/2018/10/
Visited site:  https://dinamo.md/2018/12/
Visited site:  https://dinamo.md/2019/02/
Visited site:  https://dinamo.md/2019/03/
Visited site:  https://d

In [16]:
agricultureBlogsMd = [
"https://agroexpert.md/",
"https://agrobiznes.md/",
"https://ecolocal.md/blog/",
"https://nucialtoiti.md/",
"https://agrotv.md/category/stiri/agricultura/",
"https://agromedia.md/",
"https://md.agrointel.ro/category/fermier-in-republica-moldova",
"https://www.stiriagricole.ro/"
]

agricultureArticlesMd = processUrls(100,agricultureBlogsMd,'agricultureMd.csv')

Visited site:  https://agroexpert.md/rus/dinastii/munca-inovatie-si-sprijin-valorile-dupa-care-se-conduce-familia-panfil
Visited site:  https://agroexpert.md/rus/agromenedzhment/seminar-dedicat-inovatiilor-in-irigarea-si-fertilizarea-plantatiilor
Visited site:  https://agroexpert.md/rus/agrohimiya/chem-polezna-fulivokislota-v-bake-s-glifosatom-a-guminovaya-s-mochevinoy
Visited site:  https://agroexpert.md/rus/zhivotnovodstvo/rumynskaya-poroda-korov-baltata-romaneasca-s-proizvodstvom-40-litrov-moloka
Visited site:  https://agroexpert.md/rus/agrohimiya/evropeyskie-uchenye-nashli-luchshuyu-aliternativu-glifosatu
Visited site:  https://agroexpert.md/rus/pishchevaya-promyshlennosti-10/amerikanskie-uchenye-izobreli-metod-proizvodstva-nizkokaloriynogo-sahara
Visited site:  https://agroexpert.md/rus/rastenievodstvo/cand-e-momentul-optim-pentru-semanarea-legumelor
Visited site:  https://agroexpert.md/rus/rastenievodstvo/poliskie-fermery-otchayanno-pytayutsya-sohraniti-proizvodstvo-yablok-v-etom

C:\Users\ilinc\AppData\Local\Temp\ipykernel_18076\4105644542.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(htmlText, 'html.parser')


Visited site:  https://agrobiznes.md/author/advice-consulting
Visited site:  https://agrobiznes.md/un-grup-de-pomicultori-din-moldova-in-vizita-la-adviceconsulting-italia.html
Visited site:  https://agrobiznes.md/author/corteva
Visited site:  https://agrobiznes.md/author/tatiana-caraman
Visited site:  https://agrobiznes.md/ion-cojocaru-legumicultor-dorim-sa-infiintam-un-hub-agricol.html
Visited site:  https://agrobiznes.md/ziua-fructelor-2024-start-inregistrare-expozanti.html
Visited site:  https://agrobiznes.md/
Visited site:  https://agrobiznes.md/grupuri-pe-viber
Visited site:  https://agrobiznes.md/publicitate
Visited site:  https://agrobiznes.md/parteneri
Visited site:  https://agrobiznes.md/productie-video
Visited site:  https://agrobiznes.md/academia-agtech
Visited site:  https://agrobiznes.md/conferinta-agtech
Visited site:  https://agrobiznes.md/asociatie
Visited site:  https://agrobiznes.md/podcast
Visited site:  https://agrobiznes.md/slatava-grup
Visited site:  https://agrob

In [19]:
travelBlogsMd = [
"https://nataalbot.md/category/calatorii/",
"https://travelblog.md/",
"https://wagabond.blog/",
"http://gurez.md/",
"https://orheianca.eu/",
"https://anamariaursublog.wordpress.com/",
"https://eucalatorul.net/",
"https://altblogdecalatorii.wordpress.com/"
]

travelArticlesMd = processUrls(100,travelBlogsMd,'travelMd.csv')

Visited site:  https://nataalbot.md/2017/09/26/lyon-only-sau-orasul-care-te-invita-sa-l-mananci/
Visited site:  https://nataalbot.md/2017/10/20/acasa-la-regis-marcon-bucatarul-care-daruieste-amintiri-gustoase/
Visited site:  https://nataalbot.md/2017/12/08/barcelona-fructe-de-mare-gaudi-atentie-la-portofel/
Visited site:  https://nataalbot.md/2018/09/13/noma-experience-cum-a-fost/
Visited site:  https://nataalbot.md/2018/07/30/melci-trufe-jeleu-de-vin-cina-in-patru-maini-la-castel-mimi/
Visited site:  https://nataalbot.md/2018/10/23/max-surdu-bucatarul-care-te-indragosteste-de-copenhaga/
Visited site:  https://nataalbot.md/2019/10/25/pastrav-in-frunze-de-artar-si-harbuz-cu-marar-si-miere-la-desert-la-cabane-da-cote/
Visited site:  https://nataalbot.md/2019/11/09/muzeul-barnes-foundation-la-care-chimia-dintre-arta-si-noile-tehnologii/
Visited site:  https://nataalbot.md/vreau-sa-ma-promovezi/
Visited site:  https://nataalbot.md/bio%e2%80%a6/
Visited site:  https://nataalbot.md/
Visited 

In [20]:
culturalBlogsMd = [
"https://impresiidinviatasicarti.wordpress.com/",
"https://licuricieugenia.com/",
"http://www.vitalie-vovc.com/",
"https://cartier.md/blog/",
"https://eucitesc.md/blog-full-width/page/61/",
"https://teosunny.wordpress.com/"
]

culturalArticlesMd = processUrls(100,culturalBlogsMd,'culturalMd.csv')

Only 0 articles out of 100 for https://impresiidinviatasicarti.wordpress.com/
Visited site:  https://licuricieugenia.com
Visited site:  https://licuricieugenia.com/page/2/
Visited site:  https://licuricieugenia.com/2022/10/26/%ef%b8%8f-negru-este-o-alegere/
Visited site:  https://licuricieugenia.com/2022/11/02/%ef%b8%8f-iubire-fara-inceput/
Visited site:  https://licuricieugenia.com/2022/11/04/%ef%b8%8f-toamnele-la-moldova-toamnele/
Visited site:  https://licuricieugenia.com/2023/01/30/%ef%b8%8f-tu-poti-3/
Visited site:  https://licuricieugenia.com/2023/03/03/%ef%b8%8f-primavara-mea-sunt-eu/
Visited site:  https://licuricieugenia.com/2023/03/13/%ef%b8%8f-cand-mai-tarziu-e-prea-tarziu/
Visited site:  https://licuricieugenia.com/2023/03/13/%ef%b8%8f-dor-de-tata/
Visited site:  https://licuricieugenia.com/2023/04/11/%ef%b8%8f-multumesc-pentru-fericire-doamne/
Visited site:  https://licuricieugenia.com/2023/07/09/%ef%b8%8f-ador-duminicile/
Visited site:  https://licuricieugenia.com/author/

In [21]:
parentingBlogsRo = [
"https://www.printesaurbana.ro/category/parenting",
"https://www.parentingconstient.ro/blog/",
"https://mamicaurbana.ro/",
"https://suntpitic.ro/",
"https://www.parentingineradigitala.ro/blog",
"https://www.bebelorelli.ro/blog/parenting",
"https://lumealuifram.ro/blog/",
"https://mamalapatrat.ro/blog/",
"https://uraniacremene.ro/blog/",
"https://owletcare.ro/blog/"
]

parentingArticlesRo = processUrls(100,parentingBlogsRo,'parentingRo.csv')

Visited site:  https://www.parentingconstient.ro/blog/page/12/
Visited site:  https://www.parentingconstient.ro/blog/page/2/
Visited site:  https://www.parentingconstient.ro/2023/01/10/spiritualitatea-intre-concept-si-traire/
Visited site:  https://www.parentingconstient.ro/2023/01/25/este-in-inima-noastra/
Visited site:  https://www.parentingconstient.ro/author/claudia/
Visited site:  https://www.parentingconstient.ro/2023/03/28/transpersonal/
Visited site:  https://www.parentingconstient.ro/2023/06/21/omul-sfinteste-locul/
Visited site:  https://www.parentingconstient.ro/2023/07/11/moartea-subiect-tabu/
Visited site:  https://www.parentingconstient.ro/2023/08/15/cum-ne-crestem-copiii/
Visited site:  https://www.parentingconstient.ro/author/anghel/
Visited site:  https://www.parentingconstient.ro/2023/08/31/fortareata/
Visited site:  https://www.parentingconstient.ro/blog/
Visited site:  https://www.parentingconstient.ro/coaching/
Visited site:  https://www.parentingconstient.ro/copil

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://mamicaurbana.ro/opriti-bullyingul/salvati-copiii_vizual_2/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://mamicaurbana.ro/opriti-bullyingul/salvati-copiii_vizual_5/


...

Visited site:  https://lumealuifram.ro/cum-dezvoltam-memoria-copiilor/251333210_379320217270776_45252162452401268_n/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://lumealuifram.ro/cum-dezvoltam-memoria-copiilor/251333210_282899543717180_2123898864888228115_n/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://lumealuifram.ro/cum-dezvoltam-memoria-copiilor/252793699_579805746411808_6720386230623775496_n/
Visited site:  https://lumealuifram.ro/news/
Visited site:  https://lumealuifram.ro/jucarii/trenuletul-vesel-sesame/
Visited site:  https://lumealuifram.ro/jucarii/turnul-curcubeul-jucaus/
Visited site:  https://lumealuifram.ro/jucarii/puzzle-meseria-preferata/
Visited site:  https://lumealuifram.ro/jucarii/
Visited site:  https://lumealuifram.ro/jucarii/trenuletul-lui-mos-ene/
Visited site:  https://lumealuifram.ro/jucarii/joc-de-indemanare-garderoba-papusii/
Visited site:  https://lumealuifram.ro/jucarii/carusel-pentru-bebelus-magia-cosmosului/
Visited site:  https://lumealuifram.ro/jucarii/masinuta-si-masa-de-activitati-2-in-1/
Visited site:  https://lumealuifram.ro/jucarii/cos-de-picnic-pofta-buna-de-delicii/
Visited site:  https://lumealuifram.ro/jucarii/tractor-calator-premergator/
Visited site:  https://lumealuifram.ro/jucarii/banc-de-lucru-ghiocel/
Visited site